In [2]:
import sys
import os
from algorithms.scheduling import FifoAlgorithm
from core.simulation import Simulation
import config.data as test_data 

In [3]:
os.chdir('..')
# Confirm we are in the topsim root directory - config.data has path maps relative to this 
os.getcwd()

'/home/rwb/github/topsim'

In [4]:

import simpy
import time
import os
workflow_file = None
event_file = 'sim.trace'
planning_algorithm = 'heft'
env = simpy.Environment()
tmax = 36  # for starters, we will define telescope configuration as simply number of arrays that exist
salgorithm = FifoAlgorithm()
vis = False

simulation = Simulation(env,
                       test_data.telescope_config,
                        tmax,
                        test_data.machine_config,
                        salgorithm,
                        'heft',
                        event_file,
                        vis)

In [5]:
import time
import simpy

# Setup bokeh imports
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.models import LinearColorMapper
from bokeh.models.widgets import Button, DataTable, TableColumn
from bokeh.server.server import Server
from bokeh.layouts import layout, row
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()
import logging
logger = logging.getLogger(__name__)
events = []
not_started = True
start = False

def make_document(doc):
    # source = ColumnDataSource({'x': [], 'y': [], 'color': []})
    # x = []
    # y = []

    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=0, high=1)
    button = Button(label="Start", button_type="success")
    # create three plots

    plot = {
        'time': [env.now],
        'running': [len(simulation.cluster.running_tasks)],
        'finished': [len(simulation.cluster.finished_tasks)]
    }

    plotdata = ColumnDataSource(plot)

    p = figure(plot_width=400, plot_height=400)
    p.line(x='time', y='running', legend_label='Running tasks', alpha=0.2, line_width=3, color='navy', source=plotdata)
    p.line(x='time', y='finished', legend_label='Finished tasks', alpha=0.8, line_width=2, color='orange', source=plotdata)
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Tasks'

    def update_plot():
        if env:
            updata = {
                'time': [env.now],
                'running': [len(simulation.cluster.running_tasks)],
                'finished': [len(simulation.cluster.finished_tasks)]
            }
            plotdata.stream(updata)

    """
    SETUP TABLE 
    """
    columns = [
        TableColumn(field="name", title="Name"),
        TableColumn(field="start", title="Start Time"),
        TableColumn(field="demand", title="No. Antennas"),
        TableColumn(field="duration", title="Observation Length"),
        TableColumn(field="running", title="Running Status")
    ]

    sourcedata = dict(name=[], start=[], duration=[], demand=[], running=[])
    tablesource = ColumnDataSource(data=sourcedata)
    for observation in simulation.telescope.observations:
        sourcedata['name'].append(observation.name)
        sourcedata['start'].append(observation.start)
        sourcedata['duration'].append(observation.duration)
        sourcedata['demand'].append(observation.demand)
        sourcedata['running'].append(observation.status)
    data_table = DataTable(source=tablesource, columns=columns, width=600, height=280)

    def update_table():
        if env:
            for i, observation in enumerate(simulation.telescope.observations):
                dictionary = {"running": [(i, observation.status)]}
                tablesource.patch(dictionary)

    doc.add_periodic_callback(update_plot, 500)
    doc.add_periodic_callback(update_table, 500)
#     button.on_click(update_start)

    # fig.xgrid.grid_line_color = None
    # fig.x_range.start = 0
    # fig.x_range.end = 350
    # fig.legend.orientation = "horizontal"
    # fig.legend.location = "top_center"

    doc.title = "TopSim Dashboard"
    doc.add_root(row(p, data_table))
    return doc

Loading BokehJS ...

In [6]:
# logger.debug('Opening Bokeh application on http://localhost:5071/')
show(make_document)

INFO:bokeh.server.server:Starting Bokeh server version 2.1.1 (running on Tornado 6.0.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=1002&bokeh-absolute-url=http://localhost:46569&resources=none (127.0.0.1) 27.52ms
INFO:tornado.access:101 GET /ws (127.0.0.1) 0.61ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created


In [7]:
import threading

thread = threading.Thread(target=simulation.start, args=(-1,))
thread.start()

1

INFO:core.buffer:Adding observation emu data to buffer at time 11
INFO:core.telescope:Telescope is now using 0 arrays
INFO:core.telescope:Observation dingo scheduled for 11
INFO:core.telescope:Telescope is now using 18 arrays
INFO:core.scheduler:New observation emu scheduled for processing @ Time: 12
INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=1002&bokeh-absolute-url=http://localhost:37893&resources=none (127.0.0.1) 45.89ms
INFO:core.telescope:Observation vat scheduled for 20
INFO:core.telescope:Telescope is now using 36 arrays
INFO:core.buffer:Adding observation dingo data to buffer at time 26
INFO:core.telescope:Telescope is now using 18 arrays
INFO:tornado.access:101 GET /ws (127.0.0.1) 1.53ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
INFO:core.buffer:Adding observation vat data to buffer at time 51
INFO:core.telescope:Telescope is now using 0 arrays
INFO:core.scheduler:emu finished processing @ 111